# CONFOCAL Cerebral Oximetry (cerebral oximetry index, COx) and MAPopt Analysis

In [ ]:
#Authors: Kevin FH Lee, MD, PhD and Jasmine M Khan, BSc

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics as sts
import seaborn as sns
from scipy.stats import spearmanr

#### Import csv file with MAP and rSO2 data

In [ ]:
dat = pd.read_csv() #enter file location
dat.head(10)

In [ ]:
# Obtain patient ID list
patients = dat['subject_id'].drop_duplicates()

## Set sliding window length
window = 30
print(patients)

## COx analysis

Define function for iterative correlation analysis

In [ ]:
def patient_COx_analysis(patient_id,window):
    pt_COx = []    
    pt_p = []    
    pt_MAP = []
    pt_rSO2 = []
    pt_rec = []
    subset = dat[dat['subject_id'] == patient_id]
    rec_len = len(subset)         
       
    for k in range(0,rec_len-window):               
        bp = subset['MAP'].iloc[k:k+window-1]       
        ox = subset['rSO2'].iloc[k:k+window-1]
        
        coef,p = spearmanr(bp,ox)        
        
        pt_COx.append(coef)
        pt_p.append(p)    
        pt_MAP.append(sts.mean(bp))
        pt_rSO2.append(sts.mean(ox))
    return pd.Series(pt_COx),pd.Series(pt_p),pd.Series(pt_MAP),pd.Series(pt_rSO2)

#### Iterative COx analysis through each patient

In [ ]:
output_COx = []
output_p = []
output_MAP = []
output_rSO2 = []

for patient in patients:
    COx,p_vals,MAP,rSO2 = patient_COx_analysis(patient,window)
    
    output_COx.append(COx)
    output_p.append(p_vals)
    output_MAP.append(MAP)
    output_rSO2.append(rSO2)
    
    print(patient)  

In [ ]:
## Concatenate analysis outputs COx and p-values into tables

In [ ]:
output_COx_concat = pd.concat(output_COx,axis = 1)
output_p_concat = pd.concat(output_p,axis = 1)
output_MAP_concat = pd.concat(output_MAP,axis = 1)
output_rSO2_concat = pd.concat(output_rSO2,axis = 1)

output_COx_concat.index = range(window,window+len(output_COx_concat))
output_p_concat.index = range(window,window+len(output_p_concat))
output_MAP_concat.index = range(window,window+len(output_MAP_concat))
output_rSO2_concat.index = range(window,window+len(output_rSO2_concat))

output_COx_concat.columns = patients
output_p_concat.columns = patients
output_MAP_concat.columns = patients
output_rSO2_concat.columns = patients


In [ ]:
# Document recording lengths
rec_nans = np.isnan(output_MAP_concat)
#print(rec_nans)
max = len(output_MAP_concat)
rec_lengths = max - rec_nans.sum()
#print(rec_lengths)

In [ ]:
plt.plot(output_COx_concat.loc[:,patients.iloc[0]])
plt.plot(output_p_concat.loc[:,patients.iloc[0]],'.')
plt.show()

In [ ]:
plt.plot(output_MAP_concat.loc[:,patients.iloc[0]],'b')
plt.plot(output_rSO2_concat.loc[:,patients.iloc[0]],'r')

In [ ]:
# Analyze subset of statistically significant COx 

p_thresh = 0.0001
p_thresh_mask = output_p_concat < p_thresh
COx_sig = p_thresh_mask * output_COx_concat

#p_thresh_mask.head()
#COx_sig.head()

In [ ]:
plt.plot(COx_sig.loc[:,patients.iloc[0]],'.')
#COx_sig.head()

### Examine POSITIVE COx

In [ ]:
COx_pos = (COx_sig > 0) * COx_sig
COx_pos.head()
plt.plot(COx_pos.loc[:,patients.iloc[0]],'.')

In [ ]:
# Cumulative Time with Dysfunctional CA (mins)

dysfCA_mincount = COx_pos > 0
dysfCA_mincount.head()

time_dysfCA = dysfCA_mincount.sum()
print(time_dysfCA)


In [ ]:
#Proportion of time Dysfunctional

prop_dysfCA = time_dysfCA/rec_lengths
print(prop_dysfCA)


#### Examine NEGATIVE COx only

In [ ]:
COx_neg = (COx_sig < 0) * COx_sig
#COx_neg.head()
#plt.plot(COx_pos.loc[:,patients.iloc[0]],'.')

# Cumulative Time with Dysfunctional CA
neg_mincount = COx_neg < 0
neg_mincount.head()

time_neg = neg_mincount.sum()
#print(time_neg)

prop_neg = time_neg/rec_lengths
print(prop_neg)

In [ ]:
# Create table with COx results

cox_analysis = pd.DataFrame({'recording_length': rec_lengths,'time_dysfCA': time_dysfCA, 'prop_dysfCA': prop_dysfCA})
cox_analysis.head()

## Plotting

In [ ]:
print(patients)

In [ ]:
# enter patient ID
patient = patients.iloc[0]

plt.plot(output_MAP_concat.loc[:,patient],)
plt.plot(output_rSO2_concat.loc[:,patient],'.')
plt.ylabel('MAP'), plt.xlabel('time (min)')
plt.show()

plt.plot(output_COx_concat.loc[:,patient],'.')
plt.plot(COx_pos.loc[:,patient],'ro')
plt.ylabel('COx'), plt.xlabel('time (min)')
plt.show()

plt.plot(output_p_concat.loc[:,patient],'.')
plt.ylabel('p-value'), plt.xlabel('time (min)')
plt.show()


## MAPopt Analysis


In [ ]:
#Reformat data

output_MAP_table = pd.concat(output_MAP,axis = 0, keys = patients)
print(output_MAP_table)

output_COx_table = pd.concat(output_COx,axis = 0, keys = patients)
print(output_COx_table)

In [ ]:
#Create variables

mean_cox = np.mean(output_COx_concat)
print(mean_cox)

std_cox = np.std(output_COx_concat)
print(std_cox)

neg_std_cox = - (np.std(output_COx_concat))
print(neg_std_cox)

mean_map = np.mean(output_MAP_concat)
print(mean_map)

In [ ]:
# Calculate MAPopt and MAPopt boundaries

COx_thresh_mask = []
MAP_zero = [ ]
MAPopt = []
MAPopt_SD = []
output_MAPopt = []
output_MAP_SD = []
MAPopt_lower = []
MAPopt_upper = []
output_MAPopt_lower = []
output_MAPopt_upper = []


for i in range(0,len(std_cox)):
    COx_thresh_mask = abs(output_COx_concat.iloc[:,i]) < std_cox[i]
    MAP_zero = COx_thresh_mask * output_MAP_concat.iloc[:,i]
    MAP_zero = MAP_zero.replace(0, np.NaN)
    MAPopt = np.mean(MAP_zero)
    MAPopt_SD = np.std(MAP_zero)
    MAPopt_lower = np.subtract(MAPopt, MAPopt_SD)
    MAPopt_upper = np.add(MAPopt, MAPopt_SD)
    
    str(output_MAPopt.append(MAPopt))
    str(output_MAP_SD.append(MAPopt_SD))
    str(output_MAPopt_lower.append(MAPopt_lower))
    str(output_MAPopt_upper.append(MAPopt_upper))


map_boundaries = pd.DataFrame({'subject_id': patients,'MAPopt': output_MAPopt, 'MAPopt_SD': output_MAP_SD, 'MAPopt_lower': output_MAPopt_lower, 'MAPopt_upper': output_MAPopt_upper}) 
print(map_boundaries)


In [ ]:
# Calculate area within and outside MAPopt

area_outside = []
area_map_opt = []
prop_area_outside = []

for i in patients:
    area_above = 0
    area_below = 0
    area_within = 0
    outside = 0
    prop_outside = 0
    
    subset = dat[dat['subject_id'] == i]
    rec_len = len(subset)
    MAPopt_low = map_boundaries.MAPopt_lower[map_boundaries['subject_id'] == i]
    MAPopt_low = MAPopt_low.to_numpy() 
    MAPopt_high = map_boundaries.MAPopt_upper[map_boundaries['subject_id'] == i]
    MAPopt_high = MAPopt_high.to_numpy() 
    
    for k in range(0, rec_len):
        if subset.MAP.iloc[k] > MAPopt_high:
            area_above = area_above + (subset.MAP.iloc[k] - MAPopt_high)
        elif subset.MAP.iloc[k] < MAPopt_low:
            area_below = area_below + (MAPopt_low - subset.MAP.iloc[k])
        elif subset.MAP.iloc[k] <= MAPopt_high and subset.MAP.iloc[k] >= MAPopt_low:
            area_within = area_within + (subset.MAP.iloc[k] - MAPopt_low)  
            
    outside = (area_above + area_below)
    prop_outside = (outside / (outside + area_within))
    
    area_outside.append(outside)
    area_map_opt.append(area_within)
    str(prop_area_outside.append(prop_outside))
    
    #To create figures:
    plt.figure()
    plt.plot(subset.minute, subset.MAP,'b')
    plt.axhline(y=MAPopt_low, color='g', linestyle='--')
    plt.axhline(y=MAPopt_high, color='g', linestyle='--')
    plt.ylabel('MAP (mmHg)'), plt.xlabel('Time (min)')
    
    #plt.savefig("MAPopt{i}.png".format(i=i))  #to save plots as .png
    
#plt.close()

In [ ]:
# Create table with MAPopt results

map_analysis = pd.DataFrame({'subject_id': patients,'MAP_area_outside': area_outside, 'MAP_area_within': area_map_opt, 'prop_area_outside': prop_area_outside}) 

map_analysis['MAP_area_outside'] = map_analysis['MAP_area_outside'].str[0]
map_analysis['MAP_area_within'] = map_analysis['MAP_area_within'].str[0]
map_analysis['prop_area_outside'] = map_analysis['prop_area_outside'].str[0]

map_analysis.head(32)


In [ ]:
# Create summary file

cox_map_summary = cox_analysis.merge(map_boundaries, on='subject_id', how='outer').merge(map_analysis, on='subject_id', how='outer')
cox_map_summary.head()

In [ ]:
# Save files

cox_analysis.to_csv('_cox_analysis.csv')
map_boundaries.to_csv('_map_boundaries.csv')
map_analysis.to_csv('_map_analysis.csv')
cox_map_summary.to_csv('_cox_map_summary.csv')